## Frozen CLIP CAM Refinement
To provide supervision for the prediction P in Eq. (3), we generate the pixel-level pseudo label from the initial CAM of the frozen backbone.

The frozen backbone can only provide static CAM, which means pseudo labels used as supervision cannot be improved during training. The same errors in pseudo labels lead to uncorrectable optimization in the wrong directions. Therefore, we design the Frozen CLIP CAM Refinement module (RFM) to **dynamically update CAM to improve the quality of pseudo labels**.


We first follow [Clip is also an efficient segmenter](https://arxiv.org/abs/2212.09506) to generate the initial CAM.

For the given image $I$ with its class labels, $I$ is input to the CLIP image encoder.\
The class labels are used to build text prompts and input to the CLIP text encoder.

Then, the extracted image features (after pooling) and text features are used to compute the distance and further activated by the softmax function to get the classification scores.


After that, we use GradCAM to generate the initial CAM $M_{init} \in \mathbb R^{(|C_I|+1) \times h \times w}$ where $(|C_I|+1)$ indicates all class labels in the image $I$ including the background class.

To thoroughly utilize the prior knowledge of CLIP, the CLIP model is fixed.

Although we find that such a frozen backbone can provide strong semantic features for the initial CAM with only image-level labels, as illustrated in Fig. 3(a), $M_{init}$ cannot be optimized as it is generated from the frozen backbone, limiting the quality of pseudo labels.\
Therefore, how to rectify $M_{init}$ during training becomes a key issue.

Our intuition is to **use feature relationships to rectify the initial CAM**.\
- However, we cannot directly use the attention maps from the CLIP image encoder as the feature relationship, as such attention maps are also fixed.

Nevertheless, the decoder is constantly being optimized, and we attempt to use its features to establish feature relationships to guide the selection of attention values from the CLIP image encoder, keeping useful prior CLIP knowledge and removing noisy relationships.

With more reliable feature relationships, the CAM quality can be dynamically enhanced.

In detail, we first generate an affinity map based on the feature map $F_u$ in Eq. (2) from our decoder:
$$A_f = \text{Sigmoid}(F_u^T F_u)$$
where:
- $F_u \in \mathbb R^{d \times h \times w}$ is first flattened to $\mathbb R^{d \times hw}$
- Sigmoid(·) is the sigmoid function to guarantee the range of the output is from 0 to 1.
- $A_f \in \mathbb R^{hw \times hw}$ is the affinity map.
- $T$ means matrix transpose.

Then we extract all the multi-head attention maps from the frozen CLIP image encoder, denoted as $\{A_s^l\}_{l=1}^N$ and each $A_s^l \in \mathbb R^{d \times hw \times ww}$.\

For each $A_s^l$, we use $A_f$ as a standard map to evaluate its quality:
$$S^l = \sum_{i=1}^{hw}\sum_{j=1}^{hw} |A_f(i,j) - A_s^l(i,j)|$$


We use the above $S^l$ to compute a filter for each attention map:
$$
G^l = \begin{cases}
1 & \text{if } S^l < \frac{1}{N - N_0 + 1} \sum_{l = N_0}^N S^l \\
0 & \text{else}
\end{cases}
$$
where $G^l \in \mathbb R^{1 \times 1}$, and it is expanded to $G_e^l \in \mathbb R^{hw \times hw}$ for further computation.



We use the average value of all $S^l$ as the threshold.
- If the current $S^l$ is less than the threshold, it is more reliable, and we set its filter value as 1. 
- Otherwise, we set the filter value as 0. 

Based on this rule, we keep high-quality attention maps and remove weak attention maps.


We then combine $A_f$ and the above operation to build the refining map:
$$R = \frac{A_f}{N_m} \sum_{l=1}^{N} G_e^l A_s^l$$
where $N_m$ is the number of valid $A_s^l$, i.e., $N_m = \sum_{l=N_0}^{N} G^l$


Then, following the previous apporaches, we generate the refined CAM:
$$M_f^c = \left(\frac{R_{nor} + R_{nor}^T}{2}\right)^{\alpha} \cdot M_{init}^c$$
where:
- $c$ is the specified class, 
- $M_f^c$ is the refined CAM for class $c$, 
- $R_{nor}$ is obtained from $R$ using row and column normalization ([Sinkhorn normalization](https://projecteuclid.org/journals/annals-of-mathematical-statistics/volume-35/issue-2/A-Relationship-Between-Arbitrary-Positive-Matrices-and-Doubly-Stochastic-Matrices/10.1214/aoms/1177703591.full))
- $\alpha$ is a hyperparameter.


This part passes a box mask indicator to restrict the refining region.

- $M_{init}^c$ is the CAM for class $c$ after reshaping to $\mathbb R^{hw \times 1}$


Finally, $M_f$ is input to the online post-processing module, i.e., pixel adaptive refinement module proposed in [Learning affinity from attention](https://arxiv.org/abs/2203.02664), to generate final online pseudo labels $M_p \in \mathbb R^{h \times w}$.


In this way, our RFM uses the updated feature relationship in our decoder to assess the feature relationship in the frozen backbone to select reliable relationships.\


Then, higher-quality CAM can be generated with the help of more reliable feature relationships for each image. Fig. 3 shows the detailed comparison of generated CAM using different refinement methods.\
Our method generates more accurate responses than the static refinement method proposed in [29] and the initial CAM.
